# Análise de Distribuidoras de Energias Elétricas do Brasil

## Objetivo:
O notebook atual tem como objetivo a coleta de dados no site de Dados Abertos da ANEEL para posterior análise.

## Libs:

In [2]:
! pip install --quiet requests 
! pip install --quiet pandas
! pip install --quiet numpy
import requests
import pandas as pd
import numpy as np


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\bella\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\bella\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\bella\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Coleta:

In [3]:
# Varíaveis:

# API CKAN da ANEEL:
BASE_URL = "https://dadosabertos.aneel.gov.br/api/3/action/datastore_search"

# Datasets:
kpiDECFEC = "4493985c-baea-429c-9df5-3030422c71d7" # Dados de 2019 até 2029
kpiDECFEC_old = "b4dbdc46-a8a3-4e5d-9565-9f23b1156496" # Dados de 2010 até 2019
kpiCOMPCONT = "364d945e-a18b-4111-ab1b-73aa0f7b06b1" # Dados de 2019 até 2029
kpiCOMPCONT_old = "8cdad784-07c1-4a67-a285-2f2adce5df70" # Dados de 2010 até 2019
kpiLIMITES = "fd69e1dd-fd66-4269-b60c-cc0b7eb221b4" # Limites de DEC e FEC
kpiNAETMAE = "73b00e68-66b1-4a72-8d72-b7baab47048c"
kpiDRPDRC = "b6755d51-f537-4e0f-8fd8-d2cead66178a"
kpiCOMPCONF = "476d0490-a225-4de7-89a8-bb7a189f0868"
kpiCOMERCIAL = "11473878-1059-44f8-abf0-677212ff247b"
kpiINADIMPLENCIA = "0bb73b5d-b2e1-417d-9873-6de6b2d397d1"
kpiSEGURANCA = "ebe17185-a400-4f7a-a680-87c7d67f43dc"
kpiINDQUAL_MUN = "3f841488-80a8-42f2-a6ca-e0c593b228de"
kpiSATISFACAO = "7abd3a47-1e1a-4a33-933b-48d658a6f912"
kpiRECLAMACAO_old = "6c3e074c-d9ab-4840-8c6c-7993faa36ddb" # Dados de 2010 até 2022
kpiRECLAMACAO_2023 = "426f4fb2-6a89-452a-8236-cc48153ee607"
kpiRECLAMACAO_2024 = "4af32411-da8b-492c-ae15-8f615e35d2e2"
kpiRECLAMACAO_2025 = "83839c4e-78f9-4f68-bcb7-51dcfcf05c9c"
kpiTARIFAS = "fcf2906c-7c32-4b9b-a637-054e7a5234f4"
kpiAUTOINFRACAO = "f221158a-93a3-423f-b794-4312b6985a24"
kpiTERMONOTIFICACAO = "3aec3f42-5004-44e1-8b9c-4562084196f6"

# Domínios:
dominio = "e3724332-7de8-4273-8f0d-b20ebb91bf7c"
agentes = "64250fc9-4f7a-4d97-b0d4-3c090e005e1c"

## Funções & Definições:

In [4]:
# Visualizção do Dataframe:
pd.set_option('display.max_colwidth', None) 

In [5]:
# Coletar dados na URL da ANEEL:
def coletar_dados_api(resource_id):
   # Variáveis locais:
    offset = 0
    total = []

    # Execução da função:
    while True:
        url = f"{BASE_URL}?resource_id={resource_id}&limit=10000&offset={offset}"
        resposta = requests.get(url)
        resposta.raise_for_status()    
        dados_json = resposta.json()
        
        registros = dados_json['result']['records']

        if not registros: # Encerrar se não tiver mais registros
                    break  
        
        total.extend(registros)
        offset += 10000
        print(f"Coletados {len(total)} registros.")

        # Formação do df:
        df = pd.DataFrame(registros)
    return df

def coletar_dados_csv(url_csv):
    df = pd.read_csv(url_csv, sep=";", encoding="latin1", decimal=',')

    return df

## Análise

### Definições Gerais:

In [6]:
# Dataframe de domínio:
dfDom = coletar_dados_api(dominio)

# Ajustes do df:
dfDom.columns = dfDom.columns.str.upper()
dfDom = dfDom[['SIGINDICADOR','DSCINDICADOR']]

# Visualização:
dfDom.head()

Coletados 468 registros.


,SIGINDICADOR,DSCINDICADOR
0,AREA,"Área do conj., expressa em km2, correspondente a área geogr. e não a área elétr."
1,AREAT,Área do conjunto em km2
2,CM,Encargo de uso do sistema de distribuição aplicado à unidade cons. (mensal)
3,CMA,Encargo de uso do sistema de distribuição aplicado à unidade consumidora (anual)
4,CMM,"Consumo médio mensal, em MWh, média aritmética simples do último ano, excluin-"


In [7]:
# Dataframe de agentes:
dfAg = coletar_dados_api(agentes)

# Ajustes de colunas:
dfAg.columns = dfAg.columns.str.upper()
dfAg = dfAg.rename(columns={'SIGPESSOA':'SIGAGENTE'})

# Filtro de campos:
dfAg = dfAg[(dfAg['IDCATIVO'] == 'A') & (dfAg['IDCDISTRIBUICAO'] == '1')]

# Redução do df:
dfAg = dfAg[['SIGAGENTE','NOMRAZAOSOCIAL']]

# Visualização:
dfAg.head()

Coletados 9584 registros.


,SIGAGENTE,NOMRAZAOSOCIAL
103,ANATEL,AGENCIA NACIONAL DE TELECOMUNICACOES
104,ADASA,"AGENCIA REGULADORA DE AGUAS, ENERGIA E SANEAMENTO BASICO DO DISTRITO FEDERAL - ADASA"
294,AME,AMAZONAS ENERGIA S.A
319,ENEL RJ,AMPLA ENERGIA E SERVICOS S.A.
912,None,BOQUEIRAO ENERGIA LTDA


### Análise de DEC e FEC:

#### Preparação:

In [8]:
# Geração do dataframe:
dfDECFEC = coletar_dados_csv('https://dadosabertos.aneel.gov.br/dataset/d5f0712e-62f6-4736-8dff-9991f10758a7/resource/4493985c-baea-429c-9df5-3030422c71d7/download/indicadores-continuidade-coletivos-2020-2029.csv')

In [9]:
dfDECFECT = dfDECFEC

# Ajustes de colunas:
dfDECFECT.columns = dfDECFECT.columns.str.upper() # Colunas em maiúsculo
dfDECFECT = dfDECFECT[['SIGAGENTE','IDECONJUNDCONSUMIDORAS','SIGINDICADOR','ANOINDICE','NUMPERIODOINDICE','VLRINDICEENVIADO']]

# Join com o domínio:
dfDECFECT = pd.merge(dfDECFECT, dfDom, how='left', on='SIGINDICADOR')

# Join para selecionar apenas as distribuidoras ativas em 2025:
# ! Esse item não é utilizado porque o SIGAGENTE não dá match com o da ANEEL
#dfDECFECT = pd.merge(dfDECFECT, dfAg, how='inner', on='SIGAGENTE') 

dfDECFECT.head()

,SIGAGENTE,IDECONJUNDCONSUMIDORAS,SIGINDICADOR,ANOINDICE,NUMPERIODOINDICE,VLRINDICEENVIADO,DSCINDICADOR
0,EAC,12590,FECIPC,2020,1,0.00,"FEC interrup. origem inter. ao sist distr, prog. e ocorr. em dia crítico"
1,EAC,12590,DECXNC,2020,5,0.00,"DEC interrup. origem ext. ao sist distr, não prog. e ocorr. em dia crítico"
2,CEA,14566,FEC,2020,4,1.39,Freqüência Equivalente de Interrupção por Unidade Consumidora - 3 Min.
3,ETO,13623,DEC,2020,8,0.04,Duração Equivalente de Interrupção por Unidade Consumidora - 3 Min.
4,ETO,13660,FECXN,2020,1,0.00,FEC de interrupção de origem externa ao Sist. de Dist. e não programada


#### Definições das distribuidoras por unidades consumidoras:

In [10]:
# Avaliação da quantidade de clientes por distribuidora:
# ! Ajustar sempre para o último mês disponível ou para o mês de análise desejado:
dfGrandesDiscos = dfDECFECT[(dfDECFECT['ANOINDICE'] == 2025) & (dfDECFEC['NUMPERIODOINDICE'] == 7) & (dfDECFECT['SIGINDICADOR'] == 'NumCon')][['SIGAGENTE','VLRINDICEENVIADO']].groupby(by='SIGAGENTE').sum().reset_index()

# Filtro das distribuidoras com mais de 400 mil UCs (filtro ANEEL):
dfGrandesDiscos = dfGrandesDiscos[dfGrandesDiscos['VLRINDICEENVIADO'] > 400000]

# Renomeando colunas:
dfGrandesDiscos = dfGrandesDiscos.rename(columns={'VLRINDICEENVIADO': 'NUC'})

dfGrandesDiscos.head()

,SIGAGENTE,NUC
0,AME,906769.0
5,CEEE-D,1957930.0
8,CELESC,3469262.0
10,CEMIG-D,9431438.0
44,COELBA,6257879.0


### Análise de compensação

#### Preparação:

In [11]:
# Geração do dataframe:
dfComp = coletar_dados_csv('https://dadosabertos.aneel.gov.br/dataset/d5f0712e-62f6-4736-8dff-9991f10758a7/resource/364d945e-a18b-4111-ab1b-73aa0f7b06b1/download/indicadores-continuidade-coletivos-compensacao-2020-2029.csv')

In [19]:
dfCompT = dfComp

# Ajustes de colunas:
dfCompT.columns = dfCompT.columns.str.upper() # Colunas em maiúsculo
dfCompT = dfCompT[['SIGAGENTE','IDECONJUNDCONSUMIDORAS','SIGINDICADOR','ANOINDICE','NUMPERIODOINDICE','VLRINDICEENVIADO']]

# Join com o domínio:
dfCompT = pd.merge(dfCompT, dfDom, how='left', on='SIGINDICADOR')

# Join para selecionar apenas as distribuidoras ativas em 2025:
# ! Esse item não é utilizado para compensação porque o SIGAGENTE não dá match com o da ANEEL
# dfCompT = pd.merge(dfCompT, dfAg, how='left', on='SIGAGENTE') 

dfCompT.head()

,SIGAGENTE,IDECONJUNDCONSUMIDORAS,SIGINDICADOR,ANOINDICE,NUMPERIODOINDICE,VLRINDICEENVIADO,DSCINDICADOR
0,EAC,12598,PGUGATT,2020,4,0.0,Valor pago a UGs AT por violacao dos limites de cont no trimestre
1,EAC,12589,PGUCATDC,2020,2,0.0,Valor pago as UCs AT por violacao dos limites de DICRI
2,EAC,12593,PGUGBTUDC,2020,1,0.0,Valor pago as UGs BT urbanas por violacao dos limites de DICRI
3,EAC,12596,QTUGMTNU,2020,7,0.0,Qtde de UGs MT não urbanas comp por violacao dos limites de cont no mes
4,ETO,13627,QTUGMTUA,2020,1,0.0,Qtde de UGs MT urbanas compens por violacao dos limites de cont no ano


In [20]:
# Separação dos indicadores:

## Tipologia do indicador:
dfCompT['INDICADOR'] = np.nan
dfCompT['INDICADOR'] = np.where(dfCompT['SIGINDICADOR'].astype(str).str.contains('PG'), 'DINHEIRO', dfCompT['INDICADOR'])
dfCompT['INDICADOR'] = np.where(dfCompT['SIGINDICADOR'].astype(str).str.contains('QT'), 'CLIENTES', dfCompT['INDICADOR'])

## Nível de Tensão do indicador:
dfCompT['TENSAO'] = np.nan
dfCompT['TENSAO'] = np.where(dfCompT['SIGINDICADOR'].astype(str).str.contains('BT'), 'BT', dfCompT['TENSAO'])
dfCompT['TENSAO'] = np.where(dfCompT['SIGINDICADOR'].astype(str).str.contains('AT'), 'AT', dfCompT['TENSAO'])
dfCompT['TENSAO'] = np.where(dfCompT['SIGINDICADOR'].astype(str).str.contains('MT'), 'MT', dfCompT['TENSAO'])

## Geração:
dfCompT['GERACAO'] = np.nan
dfCompT['GERACAO'] = np.where(dfCompT['SIGINDICADOR'].astype(str).str.contains('UG'), 1, dfCompT['GERACAO'])
dfCompT['GERACAO'] = np.where(dfCompT['SIGINDICADOR'].astype(str).str.contains('UC'), 0, dfCompT['GERACAO'])

## Localização (urbano ou rural):
dfCompT['URBANO'] = np.nan
dfCompT['URBANO'] = np.where(dfCompT['DSCINDICADOR'].astype(str).str.contains('urbana'), 1, dfCompT['URBANO'])
dfCompT['URBANO'] = np.where(dfCompT['DSCINDICADOR'].astype(str).str.contains('não urbana'), 0, dfCompT['URBANO'])

dfCompT.head()

,SIGAGENTE,IDECONJUNDCONSUMIDORAS,SIGINDICADOR,ANOINDICE,NUMPERIODOINDICE,VLRINDICEENVIADO,DSCINDICADOR,INDICADOR,TENSAO,GERACAO,URBANO
0,EAC,12598,PGUGATT,2020,4,0.0,Valor pago a UGs AT por violacao dos limites de cont no trimestre,DINHEIRO,AT,1.0,NaN
1,EAC,12589,PGUCATDC,2020,2,0.0,Valor pago as UCs AT por violacao dos limites de DICRI,DINHEIRO,AT,0.0,NaN
2,EAC,12593,PGUGBTUDC,2020,1,0.0,Valor pago as UGs BT urbanas por violacao dos limites de DICRI,DINHEIRO,BT,1.0,1.0
3,EAC,12596,QTUGMTNU,2020,7,0.0,Qtde de UGs MT não urbanas comp por violacao dos limites de cont no mes,CLIENTES,MT,1.0,0.0
4,ETO,13627,QTUGMTUA,2020,1,0.0,Qtde de UGs MT urbanas compens por violacao dos limites de cont no ano,CLIENTES,MT,1.0,1.0


In [21]:
# Verificação:
dfCompT[(dfCompT['INDICADOR'].isnull()) | (dfCompT['TENSAO'].isnull()) | (dfCompT['GERACAO'].isnull())]

,SIGAGENTE,IDECONJUNDCONSUMIDORAS,SIGINDICADOR,ANOINDICE,NUMPERIODOINDICE,VLRINDICEENVIADO,DSCINDICADOR,INDICADOR,TENSAO,GERACAO,URBANO


In [22]:
# Filtro das grandes distribuidoras apenas:
dfCompT = pd.merge(dfCompT, dfGrandesDiscos, how='inner', on = 'SIGAGENTE')

# Filtro para retirar as gerações:
dfCompT = dfCompT[dfCompT['GERACAO'] == 0]

In [23]:
# Visão final do dataframe:
dfCompT.head()

,SIGAGENTE,IDECONJUNDCONSUMIDORAS,SIGINDICADOR,ANOINDICE,NUMPERIODOINDICE,VLRINDICEENVIADO,DSCINDICADOR,INDICADOR,TENSAO,GERACAO,URBANO,NUC
2,ETO,13621,QTUCBTUDC,2020,8,0.0,Qtde de UCs BT urbanas comp por violacao dos limites de DICRI,CLIENTES,BT,0.0,1.0,681872.0
3,ETO,13638,QTUCATT,2020,1,0.0,Qtde de UCs AT compens por violacao dos limites de cont no trimestre,CLIENTES,AT,0.0,NaN,681872.0
4,ETO,16028,QTUCMTUT,2020,3,0.0,Qtde de UCs MT urbanas compens por violacao dos limites de cont no trimestre,CLIENTES,MT,0.0,1.0,681872.0
5,ETO,16034,PGUCMTNUT,2020,3,0.0,Valor pago a UCs MT não urbanas por violacao dos limites de cont no trimestre,DINHEIRO,MT,0.0,0.0,681872.0
6,ETO,13621,QTUCBTNUDC,2020,11,3.0,Qtde de UCs BT não urbanas comp por violacao dos limites de DICRI,CLIENTES,BT,0.0,0.0,681872.0


In [25]:
# Validação dos indicadores restantes:
dfCompT.groupby(by=['SIGINDICADOR','DSCINDICADOR']).count()

,,SIGAGENTE,IDECONJUNDCONSUMIDORAS,ANOINDICE,NUMPERIODOINDICE,VLRINDICEENVIADO,INDICADOR,TENSAO,GERACAO,URBANO,NUC
SIGINDICADOR,DSCINDICADOR,,,,,,,,,,
PGUCAT,Valor pago a UCs AT por violacao dos limites de continuidade no mes,196684,196684,196684,196684,196684,196684,196684,196684,0,196684
PGUCATA,Valor pago a UCs AT por violacao dos limites de cont no ano,5927,5927,5927,5927,5927,5927,5927,5927,0,5927
PGUCATDC,Valor pago as UCs AT por violacao dos limites de DICRI,196253,196253,196253,196253,196253,196253,196253,196253,0,196253
PGUCATT,Valor pago a UCs AT por violacao dos limites de cont no trimestre,23316,23316,23316,23316,23316,23316,23316,23316,0,23316
PGUCBTNU,Valor pago a UCs BT não urbanas por violacao dos limites de cont no mes,196829,196829,196829,196829,196829,196829,196829,196829,196829,196829
PGUCBTNUA,Valor pago a UCs BT não urbanas por violacao dos limites de cont no ano,5927,5927,5927,5927,5927,5927,5927,5927,5927,5927
PGUCBTNUDC,Valor pago as UCs BT não urbanas por violacao dos limites de DICRI,196258,196258,196258,196258,196258,196258,196258,196258,196258,196258
PGUCBTNUT,Valor pago a UCs BT não urbanas por violacao dos limites de cont no trimestre,23316,23316,23316,23316,23316,23316,23316,23316,23316,23316
PGUCBTU,Valor pago a UCs BT urbanas por violacao dos limites de continuidade no mes,196699,196699,196699,196699,196699,196699,196699,196699,196699,196699


#### Análise: